In [1]:
import math                  # math functions (especially sine)
import gc                    # garbage collector: used to pick up all objects

In [2]:
# create a dictionary of elements with their masses in AMU
elements = {'Ce':{'Mass': 140.12},
            'Co':{'Mass': 58.933},
            'Cr':{'Mass': 51.996},
            'Fe':{'Mass': 55.874},
            'Gd':{'Mass': 157.25},
            'Hf':{'Mass': 178.49},
            'La':{'Mass': 138.91},
            'Mg':{'Mass': 24.312},
            'Mn':{'Mass': 54.950},
            'Ni':{'Mass': 58.710},
            'O' :{'Mass': 15.999},
            'Sc':{'Mass': 44.956},
            'Sr':{'Mass': 87.620},
            'Ti':{'Mass': 47.900},
            'Zr':{'Mass': 91.220}}

def ang2mm(dist_ang):
    dist_mm = dist_ang*(1E-7)
    return dist_mm

def ang2mm3(vol_ang):
    vol_mm = vol_ang*(1E-21)
    return vol_mm
    
def mol2one(molar_mass):
    mass_one = molar_mass/(6.0221409E23)
    return mass_one

In [3]:
# non-object oriented volume functions
def vol_cubic(a):
    volume = a*a*a
    return volume

def vol_tetr(a,c):
    volume = a*a*c
    return volume

def vol_orth(a,b,c):
    volume = a*b*c
    return volume

def vol_monoc(a,b,c,B):
    volume = a*b*c*math.sin(B)

In [4]:
class Composition:
    
    def __init__(self, composition, fractions, crystal, quality, Z):
        # add initial values
        # composition should be a list of A and B site elements
        # fractions should be the fractions of all elements, in the order of 
        #     composition. The last element should be fraction of Oxygen
        # crystal is the crystal structure (cubic, tetragonal, etc.)
        # quality is the quality of the XRD data
        # Z is the number of formula units per unit cell
        self.comp = composition
        self.fractions = fractions
        self.crystal = crystal
        self.quality = quality
        self.Z = Z
        self.form_mass = 0
        
        # iterate through A and B site atoms to add their mass (in AMU) to total
        for i, ele in enumerate(composition):
            val = elements[ele]['Mass'] * self.fractions[i]
            self.form_mass += val
            
        # add in mass of oxygen atoms (0 for those where fraction is not given)
        self.form_mass += elements['O']['Mass'] * self.fractions[-1]
        
    def v_cube(self, a):
        self.volume = a*a*a#/(1E21)
        return self.volume
    
    def v_tetr(self, a,c):
        self.volume = a*a*c#/(1E21)
        return self.volume
    
    def v_rhom(self,V):
        # note this takes the volume directly from the XRD data
        self.volume = V#/(1E21)
        return self.volume
    
    def v_orth(self,a,b,c):
        self.volume = a*b*c#/(1E21)
        return self.volume
    
    def find_dense(self):
        vol_cm = self.volume/(1E24) # convert from angstroms^3 to cm^3
        mass_g = self.form_mass/(6.0221409E23) # convert from AMU to grams
        self.density = (mass_g*self.Z)/vol_cm # multiply by Z (formula units)
        return self.density


In [7]:
# Start creating all compositions, with their volume included
# LSCF_Cr
LaFeO3 = Composition(['La','Fe'],[1,1,3],'cubic','calculated', 1)
LaFeO3.v_cube(a=3.89)

#Cr101LaOx = Composition(['Cr','La'],[1.01,1,0],'cubic','indexed', 1) # note that z value was not given, 
# nor was an x (oxygen percent)
#Cr101LaOx.v_cube(a=3.884)

LaCrO3_cube = Composition(['La','Cr'],[1,1,3],'cubic','calculated', 1)
LaCrO3_cube.v_cube(a=3.880) # a = 3.900 from other calculated source

SrFeO283 = Composition(['Sr','Fe'],[1,1,2.83],'tetragonal','indexed', 1)
SrFeO283.v_tetr(a=3.864,c=3.853)

LaCr04Fe06O3 = Composition(['La','Cr','Fe'],[1,0.4,0.6,3],'cubic','indexed', 1)
LaCr04Fe06O3.v_cube(a=3.885)

#LSCF_Cr_Mg
La05Sr05FeO3 = Composition(['La','Sr','Fe'],[0.5,0.5,1,3],'rhombo_h','calculated', 6)
La05Sr05FeO3.v_rhom(V = 352.87)

#LSCF_Cr_Prax
La03Sr07FeO3 = Composition(['La','Sr','Fe'],[0.3,0.7,1,3],'rhombo_h','calculated', 6)
La03Sr07FeO3.v_rhom(V = 348.16)

La075Sr025CrO3 = Composition(['La','Sr','Cr'],[0.75,0.25,1,3],'rhombo_h','calculated', 6)
La075Sr025CrO3.v_rhom(V = 347.55)

# LSCFP995_FCM
#SrFeO286 = Composition(['Sr','Fe'],[1,1,2.86],'tetragonal','indexed', 1) # note that z was not given, 
# and density is broken for some reason
#SrFeO286.v_tetr(a=10.934,c=7.705)

# LSCrF
LaCrO3_orth = Composition(['La','Cr'],[1,1,3],'orthrhombic','calculated', 4)
LaCrO3_orth.v_orth(a=5.514, b=5.478, c=7.752) 

La0985CrO29775 = Composition(['La','Cr'],[0.985,1,2.9775],'orthrhombic','calculated', 4)
La0985CrO29775.v_orth(a=5.475, b=7.754, c=5.513) 

# LSCrF_Mg
La06Sr04FeO3 = Composition(['La','Sr','Fe'],[0.6,0.4,1,3],'rhombo_h','calculated', 6)
La06Sr04FeO3.v_rhom(V=355.09)

MgCrO4 = Composition(['Mg','Cr'],[1,1,4],'orthrhombic','indexed', 4)
MgCrO4.v_orth(a=5.497, b=8.368, c=6.333)

MgCr2O4 = Composition(['Mg','Cr'],[1,2,4],'orthrhombic','indexed', 8)
MgCr2O4.v_cube(a=8.333)

# LSF20_FCM
La08Sr02FeO3 = Composition(['La','Sr','Fe'],[0.8,0.2,1,3],'orthrhombic','calculated', 4)
La08Sr02FeO3.v_orth(a=5.532, b=5.533 ,c=7.835)

print('')

In [10]:
# iterate through objects created above and calculate their densities (in g/cm^3)
for obj in gc.get_objects():
    if isinstance(obj, Composition):
        obj.find_dense()
        print(list(obj.__dict__.values())[0:3],'-->',obj.density,'g/cm^3')

[['La', 'Fe'], [1, 1, 3], 'cubic'] --> 6.848807812740736 g/cm^3
[['La', 'Cr'], [1, 1, 3], 'cubic'] --> 6.791653396509335 g/cm^3
[['Sr', 'Fe'], [1, 1, 2.83], 'tetragonal'] --> 5.448933536230788 g/cm^3
[['La', 'Cr', 'Fe'], [1, 0.4, 0.6, 3], 'cubic'] --> 6.831356892191375 g/cm^3
[['La', 'Sr', 'Fe'], [0.5, 0.5, 1, 3], 'rhombo_h'] --> 6.1308040214412065 g/cm^3
[['La', 'Sr', 'Fe'], [0.3, 0.7, 1, 3], 'rhombo_h'] --> 5.920191747804192 g/cm^3
[['La', 'Sr', 'Cr'], [0.75, 0.25, 1, 3], 'rhombo_h'] --> 6.481061611544368 g/cm^3
[['La', 'Cr'], [1, 1, 3], 'orthrhombic'] --> 6.776853987104232 g/cm^3
[['La', 'Cr'], [0.985, 1, 2.9775], 'orthrhombic'] --> 6.710698063846144 g/cm^3
[['La', 'Sr', 'Fe'], [0.6, 0.4, 1, 3], 'rhombo_h'] --> 6.236385826441699 g/cm^3
[['Mg', 'Cr'], [1, 1, 4], 'orthrhombic'] --> 3.199058891316956 g/cm^3
[['Mg', 'Cr'], [1, 2, 4], 'orthrhombic'] --> 4.414832320090537 g/cm^3
[['La', 'Sr', 'Fe'], [0.8, 0.2, 1, 3], 'orthrhombic'] --> 6.440108051283767 g/cm^3
